<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# DKN : Deep Knowledge-Aware Network for News Recommendation
DKN \[1\] is a deep learning model which incorporates information from knowledge graph for better news recommendation. Specifically, DKN uses TransX \[2\] method for knowledge graph representaion learning, then applies a CNN framework, named KCNN, to combine entity embedding with word embedding and generate a final embedding vector for a news article. CTR prediction is made via an attention-based neural scorer. 
<img src="https://recodatasets.blob.core.windows.net/kdd2020/images%2FDKN-introduction-pic.JPG" width="600">

## Properties of DKN:
- DKN is a content-based deep model for CTR prediction rather than traditional ID-based collaborative filtering. 
- It makes use of knowledge entities and common sense in news content via joint learning from semantic-level and knnowledge-level representations of news articles.
- DKN uses an attention module to dynamically calculate a user's aggregated historical representaition.



## Data format:
### DKN takes several files as input as follows:
- training / validation / test files: each line in these files represents one instance. Impressionid is used to evaluate performance within an impression session, so it is only used when evaluating, you can set it to 0 for training data. The format is : <br> 
`[label] [userid] [CandidateNews]%[impressionid] `<br> 
e.g., `1 train_U1 N1%0` <br> 
- user history file: each line in this file represents a users' click history. You need to set his_size parameter in config file, which is the max number of user's click history we use. We will automatically keep the last his_size number of user click history, if user's click history is more than his_size, and we will automatically padding 0 if user's click history less than his_size. the format is : <br> 
`[Userid] [newsid1,newsid2...]`<br>
e.g., `train_U1 N1,N2` <br> 
- document feature file:
It contains the word and entity features of news. News article is represented by (aligned) title words and title entities. To take a quick example, a news title may be : Trump to deliver State of the Union address next week , then the title words value may be CandidateNews:34,45,334,23,12,987,3456,111,456,432 and the title entitie value may be: entity:45,0,0,0,0,0,0,0,0,0. Only the first value of entity vector is non-zero due to the word Trump. The title value and entity value is hashed from 1 to n(n is the number of distinct words or entities). Each feature length should be fixed at k(doc_size papameter), if the number of words in document is more than k, you should truncate the document to k words, and if the number of words in document is less than k, you should padding 0 to the end. 
the format is like: <br> 
`[Newsid] [w1,w2,w3...wk] [e1,e2,e3...ek]`
- word embedding/entity embedding/ context embedding files: These are npy files of pretrained embeddings. After loading, each file is a [n+1,k] two-dimensional matrix, n is the number of words(or entities) of their hash dictionary, k is dimension of the embedding, note that we keep embedding 0 for zero padding. 
In this experiment, we used GloVe\[4\] vectors to initialize the word embedding. We trained entity embedding using TransE\[2\] on knowledge graph and context embedding is the average of the entity's neighbors in the knowledge graph.<br>

## Global settings and imports

In [1]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import *
from reco_utils.recommender.deeprec.models.dkn import *
from reco_utils.recommender.deeprec.io.dkn_iterator import *
import papermill as pm
import time

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

## data paths

In [2]:
data_path = 'data_folder/my/DKN-training-folder'
tag = 'medium'  # small or full
yaml_file = './dkn.yaml' #  os.path.join(data_path, r'../../../../../../dkn.yaml')
train_file = os.path.join(data_path, r'train_{0}.txt'.format(tag))
valid_file = os.path.join(data_path, r'valid_{0}.txt'.format(tag))
test_file = os.path.join(data_path, r'test_{0}.txt'.format(tag))
user_history_file = os.path.join(data_path, r'user_history_{0}.txt'.format(tag))
news_feature_file = os.path.join(data_path, r'../paper_feature.txt')
wordEmb_file = os.path.join(data_path, r'word_embedding.npy')
entityEmb_file = os.path.join(data_path, r'entity_embedding.npy')
contextEmb_file = os.path.join(data_path, r'context_embedding.npy')
infer_embedding_file = os.path.join(data_path, r'infer_embedding_{0}.txt'.format(tag))
    

## Create hyper-parameters

In [3]:
epoch=10
hparams = prepare_hparams(yaml_file,
                          news_feature_file = news_feature_file,
                          user_history_file = user_history_file,
                          wordEmb_file=wordEmb_file,
                          entityEmb_file=entityEmb_file,
                          contextEmb_file=contextEmb_file,
                          epochs=epoch,
                          is_clip_norm=True,
                          max_grad_norm=0.5,
                          history_size=20,
                          MODEL_DIR=os.path.join(data_path, 'save_models'),
                          learning_rate=0.002,
                          embed_l2=0.0,
                          layer_l2=0.0,
                          use_entity=True,
                          use_context=True
                         )
hparams.values

<bound method HParams.values of HParams([('DNN_FIELD_NUM', None), ('EARLY_STOP', 100), ('FEATURE_COUNT', None), ('FIELD_COUNT', None), ('L', None), ('MODEL_DIR', 'data_folder/my/DKN-training-folder/save_models'), ('PAIR_NUM', None), ('SUMMARIES_DIR', None), ('T', None), ('activation', ['sigmoid']), ('att_fcn_layer_sizes', None), ('attention_activation', 'relu'), ('attention_dropout', 0.0), ('attention_layer_sizes', 32), ('attention_size', None), ('batch_size', 100), ('cate_embedding_dim', None), ('cate_vocab', None), ('contextEmb_file', 'data_folder/my/DKN-training-folder/context_embedding.npy'), ('cross_activation', 'identity'), ('cross_l1', 0.0), ('cross_l2', 0.0), ('cross_layer_sizes', None), ('cross_layers', None), ('data_format', 'dkn'), ('decay', None), ('dilations', None), ('dim', 32), ('doc_size', 15), ('dropout', [0.0]), ('dtype', 32), ('embed_l1', 0.0), ('embed_l2', 0.0), ('embed_size', None), ('embedding_dropout', 0.3), ('enable_BN', False), ('entityEmb_file', 'data_folder/m

In [4]:
input_creator = DKNTextIterator

## Train the DKN model
<img src="https://recodatasets.blob.core.windows.net/kdd2020/images%2FDKN-main.JPG" width="600">

In [5]:
model = DKN(hparams, input_creator)

In [6]:
t01 = time.time()
print(model.run_eval(valid_file))
t02 = time.time()
print((t02-t01)/60)

{'auc': 0.494, 'group_auc': 0.4935, 'mean_mrr': 0.4495, 'ndcg@2': 0.3169, 'ndcg@4': 0.5066, 'ndcg@6': 0.5842}
0.682484503587087


In [7]:
model.fit(train_file, valid_file, test_file)

step 10000 , total_loss: 0.3140, data_loss: 0.3140
at epoch 1
train info: logloss loss:0.3147331560381935
eval info: auc:0.8978, group_auc:0.8881, mean_mrr:0.8217, ndcg@2:0.8202, ndcg@4:0.8648, ndcg@6:0.8674
test info: auc:0.891, group_auc:0.882, mean_mrr:0.5442, ndcg@2:0.4684, ndcg@4:0.5759, ndcg@6:0.6194
at epoch 1 , train time: 345.9 eval time: 102.6
step 10000 , total_loss: 0.3561, data_loss: 0.3561
at epoch 2
train info: logloss loss:0.275244023245576
eval info: auc:0.9145, group_auc:0.906, mean_mrr:0.8483, ndcg@2:0.8497, ndcg@4:0.885, ndcg@6:0.8872
test info: auc:0.9084, group_auc:0.9018, mean_mrr:0.6013, ndcg@2:0.5389, ndcg@4:0.6361, ndcg@6:0.672
at epoch 2 , train time: 342.5 eval time: 107.7
step 10000 , total_loss: 0.3237, data_loss: 0.3237
at epoch 3
train info: logloss loss:0.2648009327709698
eval info: auc:0.9192, group_auc:0.9125, mean_mrr:0.857, ndcg@2:0.8603, ndcg@4:0.8917, ndcg@6:0.8938
test info: auc:0.9146, group_auc:0.9075, mean_mrr:0.6135, ndcg@2:0.5556, ndcg@4:0.6

Now we can test again the performance on valid set:

In [8]:
res = model.run_eval(valid_file)
print(res)
pm.record("res", res)

{'auc': 0.9154, 'group_auc': 0.9057, 'mean_mrr': 0.8476, 'ndcg@2': 0.8489, 'ndcg@4': 0.8848, 'ndcg@6': 0.8867}


/home/jialia/.conda/envs/reco_gpu_kdd/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
t01 = time.time()
print(model.run_eval(test_file))
t02 = time.time()
print((t02-t01)/60)

{'auc': 0.9079, 'group_auc': 0.8983, 'mean_mrr': 0.5936, 'ndcg@2': 0.5269, 'ndcg@4': 0.6259, 'ndcg@6': 0.6642}
1.027146820227305


## Document embedding inference API
After training, you can get document embedding through this document embedding inference API. The input file format is same with document feature file. The output file fomrat is: `[Newsid] [embedding]`

In [10]:
model.run_get_embedding(news_feature_file, infer_embedding_file)

## Reference
\[1\] Wang, Hongwei, et al. "DKN: Deep Knowledge-Aware Network for News Recommendation." Proceedings of the 2018 World Wide Web Conference on World Wide Web. International World Wide Web Conferences Steering Committee, 2018.<br>
\[2\] Knowledge Graph Embeddings including TransE, TransH, TransR and PTransE. https://github.com/thunlp/KB2E <br>
\[3\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[4\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/